In [24]:
from importlib import reload

import matplotlib.pyplot as plt

import torch
from PIL import Image
import torchvision.transforms as T

import multicrop
import constants
import data
import tokenizer
from datasets import emoreact, enterface
reload(multicrop)
reload(data)
reload(constants)
reload(tokenizer)
reload(emoreact)
reload(enterface)
from data import load_data 
from tokenizer import save_video_frames_tensors
from constants import *



BATCH_SZ = 3
SPLIT = [1, 0.0, 0.0]
dataset_to_use = emoreact




In [26]:

train_dl, val_dl, test_dl  = load_data(dataset_to_use, 
                                       batch_sz=BATCH_SZ,
                                       train_val_test_split=SPLIT)
def get_image_seq(crops, i, rgb=True):

    if rgb:
        image_seq = [ 
            crops[i][j].reshape(FRAMES, CHANS, HEIGHT, WIDTH)[k]
                for j in range(BATCH_SZ) for k in range(3)
        ]

    else:
        image_seq = [
            crops[i][j].reshape(CHANS, NUM_MELS, dataset_to_use.MAX_SPEC_SEQ_LEN)
                for j in range(BATCH_SZ)
        ]

    return image_seq


for data in train_dl:
    crops = data["student_spec"]
    crops = data["teacher_spec"]
    crops = data["teacher_rgb"]
    global_crop_seq = []
    for i in range(len(crops)):
        image_seq_rgb = get_image_seq(crops, i, rgb=True)
        # stacked_frames = torch.cat(image_seq_rgb, dim=0)
        # global_crop_seq.append(stacked_frames)
        global_crop_seq  += image_seq_rgb
    global_stacked = torch.cat(global_crop_seq, dim=0)
    break

    # for video, audio in zip(data["teacher_rgb"], data["teacher_spec"]):
    #     video_frames = video[0].reshape(FRAMES, CHANS, HEIGHT, WIDTH)
    #     audio_frame = audio[0]
    #     break
    # break
global_stacked = global_stacked.reshape(-1, BATCH_SZ, HEIGHT, WIDTH)
print(global_stacked.shape)

save_video_frames_tensors(global_stacked, "frames_v")

# save_video_frames_tensors(audio_frame, "frame_a")
# save_video_frames_tensors(video_frames, "frames_v")
    # total_items = len(global_crops) + len(local_crops)
# stacked_local = torch.cat(local_crops, dim=2)
# plt.imshow(global_stacked.swapaxes(0, 2).swapaxes(0, 1))
# plt.imshow(global_stacked)
# plt.imshow(stacked_local.swapaxes(0, 2).swapaxes(0, 1))

/root/depmbt/data.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return self.dataset.iloc[item][0], [self.dataset.iloc[item][i] for i in range(1, self.constants.NUM_LABELS + 1)]
/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/root/miniconda/envs/depmbt/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


torch.Size([18, 3, 224, 224])
